In [40]:
# install necessary packages
# !pip install yfinance pandas numpy scikit-learn matplotlib ipywidgets

In [41]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
import ipywidgets as widgets
from IPython.display import display

plt.style.use('fivethirtyeight')

In [42]:
from sklearn.metrics import recall_score 

class StockPredictor:
    def __init__(self, Stock_Name):
        self.Stock_Name = Stock_Name.upper()
        self.model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=42)
        self.data = None
        self.test_data = None
    
    def fetch_data(self, period="5y"):
        try:
            stock = yf.Stock_Name(self.Stock_Name)
            df = stock.history(period=period)
            if df.empty: return None
            df = df[["Close", "Open", "High", "Low", "Volume"]].copy()
            if "Volume" not in df.columns or df["Volume"].sum() == 0:
                df["Volume"] = 1 
            return df
        except Exception: return None

    def add_features(self, df):
        df = df.copy()
        df["SMA_10"] = df["Close"].rolling(window=10).mean()
        df["SMA_50"] = df["Close"].rolling(window=50).mean()
        
        delta = df["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        df["RSI"] = 100 - (100 / (1 + rs))
        
        ema_12 = df["Close"].ewm(span=12, adjust=False).mean()
        ema_26 = df["Close"].ewm(span=26, adjust=False).mean()
        df["MACD"] = ema_12 - ema_26
        
        df["Vol_SMA"] = df["Volume"].rolling(5).mean()
        df["Vol_Ratio"] = df["Volume"] / df["Vol_SMA"]
        
        df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
        df.dropna(inplace=True)
        return df

    def run_analysis(self, threshold=0.45): 
        raw_df = self.fetch_data()
        if raw_df is None: return None
        
        df = self.add_features(raw_df)
        
        predictors = ["SMA_10", "SMA_50", "RSI", "MACD", "Vol_Ratio"]
        split = int(len(df) * 0.8)
        train = df.iloc[:split]
        test = df.iloc[split:]
        self.test_data = test.copy() 
        
        self.model.fit(train[predictors], train["Target"])
        
        #  LOGIC START 
        probs = self.model.predict_proba(test[predictors])[:, 1]
        preds = (probs >= threshold).astype(int) 
        
        precision = precision_score(test["Target"], preds)
        recall = recall_score(test["Target"], preds) 
        # LOGIC END 
        
        # Predict Tomorrow
        last_row = df.iloc[[-1]][predictors]
        tomorrow_prob = self.model.predict_proba(last_row)[0][1]
        tomorrow_pred = 1 if tomorrow_prob >= threshold else 0
        
        return precision, recall, tomorrow_pred, tomorrow_prob

In [43]:
def plot_results(predictor):
    test_df = predictor.test_data
    if test_df is None: return

    # Recalculate predictions for the test set
    predictors = ["SMA_10", "SMA_50", "RSI", "MACD", "Vol_Ratio"]
    preds = predictor.model.predict(test_df[predictors])

    # Calculate Returns
    test_df["Market_Return"] = test_df["Close"].pct_change()
    test_df["Strategy_Return"] = test_df["Market_Return"] * pd.Series(preds, index=test_df.index).shift(1)

    # Cumulative Returns
    test_df["Cum_Market"] = (1 + test_df["Market_Return"]).cumprod()
    test_df["Cum_Strategy"] = (1 + test_df["Strategy_Return"]).cumprod()

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(test_df["Cum_Market"], label="Buy & Hold (Market)", alpha=0.6)
    plt.plot(test_df["Cum_Strategy"], label="AI Strategy", color='green')
    plt.title(f"Backtest: {predictor.Stock_Name} (AI vs Market)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [44]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def on_analyze_click(Stock_Name):
    # 1. Clearing the previous output 
    clear_output(wait=True)
    
    print(f"\n🧠 Analyzing {Stock_Name}...")
    
    bot = StockPredictor(Stock_Name)
    result = bot.run_analysis(threshold=0.45)
    
    if result:
        precision, recall, pred, prob = result
        
        # SECTION A: ORIGINAL STATS
        print("-" * 30)
        print(f"📊 Model Precision: {precision:.2%}")
        print(f"🎣 Model Recall:    {recall:.2%}")
        print(f"🔮 Prediction:      {'🟢 UP' if pred == 1 else '🔴 DOWN'}")
        print(f"⚖️ Confidence:      {prob:.1%}")
        print("-" * 30)
        
        # SECTION B: ENGLISH EXPLANATION 
        if pred == 1:
            signal = "BUY"
            if prob > 0.55:
                advice = "The AI is confident. This looks like a solid opportunity."
            else:
                advice = "The AI suggests a Buy, but it is a risky/weak signal."
            
            english_summary = (f"👉 VERDICT: You should *{signal}* this stock.\n"
                               f"   Reason: The model sees a {prob:.1%} chance of profit.\n"
                               f"   History: When the AI says buy, it is correct {precision:.0%} of the time.")
        else:
            signal = "SELL / WAIT"
            advice = "The AI does not see a profit opportunity tomorrow."
            english_summary = (f"👉 VERDICT: You should *{signal}* (Stay in Cash).\n"
                               f"   Reason: The model thinks the price will likely drop or stay flat.\n"
                               f"   Advice: Save your capital for a better setup.")

        print(f"💡 {advice}")
        print(english_summary)
        print("-" * 30)
        
        # 3. Plotting the graph
        plot_results(bot)
    else:
        print("❌ Error: Could not fetch data. Check the Stock_Name symbol.")

# Creating the widget
manual_widget = widgets.interact_manual(on_analyze_click, Stock_Name="RELIANCE.NS");

# Renaming the button
manual_widget.widget.children[1].description = 'Run AI Analysis'



interactive(children=(Text(value='RELIANCE.NS', continuous_update=False, description='Stock_Name'), Button(des…